# Parallel

The executed version of this tutorial is at https://elephant.readthedocs.io/en/latest/tutorials/parallel.html

`elephant.parallel` module provides a simple interface to parallelize multiple calls to any user-specified function. The typical use case is calling a function many times with different parameters.

## Available executors

`elephant.parallel` has 3 interfaces to choose from, depending whether the user has a laptop/PC or the computation is being done on a cluster machine with many nodes and MPI installed.

* `ProcessPoolExecutor` is a wrapper of python built-in `concurrent.futures.ProcessPoolExecutor`. This is recommended to run on laptops and personal computers;
* `MPIPoolExecutor` is a wrapper of `mpi4py.futures.MPIPoolExecutor`. This is recommened to run on cluster nodes with MPI-2 installed;
* `MPICommExecutor` is a wrapper of `mpi4py.futures.MPICommExecutor`. This is a legacy MPI-1 class for `MPIPoolExecutor` and is recommended to run only on cluster nodes which do not support MPI-2 protocol.

Besides these three, a `SingleProcess` executor is available as a fall-back option to test executions in a single process (no speedup).

All listed above classes has the same API and can be used interchangeably.

## How to use

Let's say you want to call some function `my_function()` for each element in a list `iterables_list` like so:

(eq. 1) `results = [my_function(arg) for arg in iterables_list]`.

If `my_function`'s implementaion does not use parallelization, you can obtain the results by computing `my_function()` assynchronously for each element in arguments list. Then the result of eq. 1 is equivalent to

(eq. 2) `results = Executor().execute(my_function, iterables_list)`,

where `Executor` can be any of listed above available executors. For more information about parallel executors in python refer to https://docs.python.org/3/library/concurrent.futures.html.

## Examples

### Example 1. Computing the mean firing rate

`mean_firing_rate()` function in `elephant.statistics` works with one spiketrain as input. Let's parallelize it by computing firing rates of 8 random spiketrains.

In [1]:
import numpy as np
import quantities as pq

from elephant.parallel import SingleProcess, MPIPoolExecutor, ProcessPoolExecutor, MPICommExecutor
from elephant.spike_train_generation import homogeneous_poisson_process
from elephant.statistics import mean_firing_rate, time_histogram

In [2]:
rate = 10 * pq.Hz
spiketrains = [homogeneous_poisson_process(rate, t_stop=10*pq.s) for _ in range(8)]

We start with a sanity check by computing the mean firing rate of the spiketrains with `SingleProcess` executor, which is run in the main process with no parallelization.

In [3]:
firing_rate0 = SingleProcess().execute(mean_firing_rate, spiketrains)
firing_rate0

[array(11.3) * 1/s,
 array(9.3) * 1/s,
 array(9.9) * 1/s,
 array(9.8) * 1/s,
 array(11.6) * 1/s,
 array(9.8) * 1/s,
 array(9.1) * 1/s,
 array(10.5) * 1/s]

Let's verify that all three other executors produce the same result, but now with parallelization turned on.

In [4]:
firing_rate1 = ProcessPoolExecutor().execute(mean_firing_rate, spiketrains)
firing_rate1

[array(11.3) * 1/s,
 array(9.3) * 1/s,
 array(9.9) * 1/s,
 array(9.8) * 1/s,
 array(11.6) * 1/s,
 array(9.8) * 1/s,
 array(9.1) * 1/s,
 array(10.5) * 1/s]

In [5]:
firing_rate2 = MPIPoolExecutor().execute(mean_firing_rate, spiketrains)
firing_rate2

[array(11.3) * 1/s,
 array(9.3) * 1/s,
 array(9.9) * 1/s,
 array(9.8) * 1/s,
 array(11.6) * 1/s,
 array(9.8) * 1/s,
 array(9.1) * 1/s,
 array(10.5) * 1/s]

In [6]:
firing_rate3 = MPICommExecutor().execute(mean_firing_rate, spiketrains)
firing_rate3

[array(11.3) * 1/s,
 array(9.3) * 1/s,
 array(9.9) * 1/s,
 array(9.8) * 1/s,
 array(11.6) * 1/s,
 array(9.8) * 1/s,
 array(9.1) * 1/s,
 array(10.5) * 1/s]

All executors produce identical output, as intended.

#### Note about MPI executors

If you print the detailed information about either of MPI executors, you will notice in this notebook that they use only one node (core) which is equivalent to `SingleProcess` executor.

In [7]:
print(MPIPoolExecutor())

MPIPoolExecutor(max_workers=None, {'maxprocs': '1', 'soft': '1', 'host': 'jupyter-inm-2d6-2delephant-2dobaqmg5v', 'arch': 'x86_64', 'thread_level': 'MPI_THREAD_MULTIPLE', 'ompi_np': '1'})


This is because both MPI executors require `-m mpi4py.futures` flag while running python scripts:

```
mpiexec -n numprocs python -m mpi4py.futures pyfile [arg] ...
```

If you run it without `mpiexec` command, as in this notebook, MPI features will still be available but only with one single core (`maxprocs=1`). For more information of how to launch MPI processes in python refer to https://mpi4py.readthedocs.io/en/stable/mpi4py.futures.html#command-line.

### Example 2. Custom functions and positional argument

Sometimes you might want to iterate over the second (or third, etc.) argument of a function. To do this, you need to create a custom function that passes its first input argument into the right position of the original function. Below is an example of how to compute time histograms of spiketrains with different `binsize`s (the second argument).

In [8]:
# step 1: initialize the first argument - spiketrains
spiketrains = [homogeneous_poisson_process(rate, t_stop=10*pq.s) for _ in range(8)]

# step 2: define your custom function
def my_custom_function(binsize):
    # specify all custom key-word options here
    return time_histogram(spiketrains, binsize, output='counts')

In [9]:
binsize_list = np.linspace(0.1, 1, num=8) * pq.s

time_hist = ProcessPoolExecutor().execute(my_custom_function, binsize_list)

/srv/conda/envs/notebook/lib/python3.7/site-packages/elephant/conversion.py:463: UserWarning: Binning discarded 20 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
/srv/conda/envs/notebook/lib/python3.7/site-packages/elephant/conversion.py:463: UserWarning: Binning discarded 14 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
/srv/conda/envs/notebook/lib/python3.7/site-packages/elephant/conversion.py:463: UserWarning: Binning discarded 14 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
/srv/conda/envs/notebook/lib/python3.7/site-packages/elephant/conversion.py:463: UserWarning: Binning discarded 27 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
/srv/conda/envs/notebook/lib/python3.7/site-packages/elephant/conversion.py:463: UserWarning: Binning discarded 33 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
/srv/conda/envs/notebook/lib/python3.7/site-packages/elephant/con

`time_hist` contains 8 AnalogSignals - one AnalogSignal per `binsize` from `binsize_list`.

### Benchmark

Finally, let's see if `ProcessPoolExecutor` brings any speedup, compared to sequential processing, to Example 2.

In [10]:
import warnings
warnings.filterwarnings("ignore")

# initialize the iteration list
binsize_list = np.linspace(0.1, 1, 100) * pq.s

In [11]:
# sequential processing
%timeit [time_histogram(spiketrains, binsize) for binsize in binsize_list]

951 ms ± 25.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# with parallelization
%timeit ProcessPoolExecutor(max_workers=4).execute(my_custom_function, binsize_list)

2.27 s ± 90.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
